<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/Score_pos_neg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,396 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [602 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,964 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bion

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("word2vec").getOrCreate()
MAX_MEMORY='5g'
spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [4]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)

# Show DataFrame
# df.show()

In [6]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string]>

In [7]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')
df = df.filter(regexp_extract(col('taste'), pattern, 0) != '')
df = df.filter(regexp_extract(col('feel'), pattern, 0) != '')
df = df.filter(regexp_extract(col('overall'), pattern, 0) != '')
df = df.filter(regexp_extract(col('score'), pattern, 0) != '')

In [ ]:
# confirm look, smell, taste, overall, and score
# df.show()

In [ ]:
# df.count()

In [8]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))
df = df.withColumn('smell_numeric', df['smell'].cast('float'))
df = df.withColumn('taste_numeric', df['taste'].cast('float'))
df = df.withColumn('feel_numeric', df['feel'].cast('float'))
df = df.withColumn('overall_numeric', df['overall'].cast('float'))
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [ ]:
df.printSchema

In [9]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'

In [10]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [11]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
# beer_df.show()

In [12]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df = text_df.filter('clean_text != "0"')

In [13]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [124]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)
final_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|          Final_text|          token_text|            features|
+-------+----------------+----------+--------------------+----+-----+-----+----+--

In [125]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [126]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('mild', 0.6426884531974792),
 ('faint', 0.6223998665809631),
 ('slight', 0.6191715002059937),
 ('mostly', 0.6169756650924683),
 ('medium', 0.6095581650733948)]

In [127]:
final_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|          Final_text|          token_text|            features|
+-------+----------------+----------+--------------------+----+-----+-----+----+--

In [128]:
final_df = final_df.select('beer_id', 'look_numeric', 'smell_numeric', 'taste_numeric', 'feel_numeric', 'overall_numeric', 'score_numeric', 'Final_text', 'features')

In [129]:
final_df.show()

+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+
|beer_id|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          Final_text|            features|
+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+
| 271781|         4.0|          4.0|          4.0|        4.25|            4.0|         4.03|MotorbreathImperi...|[motorbreathimper...|
| 184647|         4.0|          4.0|          3.5|         3.5|           3.75|          3.7|PaybackPilsner Bo...|[paybackpilsner, ...|
| 184647|         4.0|          4.0|         4.25|        3.75|            4.0|         4.08|PaybackPilsner Bo...|[paybackpilsner, ...|
| 184647|        3.25|         3.25|          3.5|         3.5|            3.5|         3.43|PaybackPilsner Bo...|[paybackpilsner, ...|
| 184647|        3.75|         3.75|          3.

In [130]:
# from pyspark.sql import functions as f
import pyspark.sql.functions as f

score_df = final_df.withColumn('class', f.when(f.col('score_numeric') > 3.5, "Positive").otherwise("Negative"))
score_df.show()

+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------+
|beer_id|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          Final_text|            features|   class|
+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------+
| 271781|         4.0|          4.0|          4.0|        4.25|            4.0|         4.03|MotorbreathImperi...|[motorbreathimper...|Positive|
| 184647|         4.0|          4.0|          3.5|         3.5|           3.75|          3.7|PaybackPilsner Bo...|[paybackpilsner, ...|Positive|
| 184647|         4.0|          4.0|         4.25|        3.75|            4.0|         4.08|PaybackPilsner Bo...|[paybackpilsner, ...|Positive|
| 184647|        3.25|         3.25|          3.5|         3.5|            3.5|         3.43|PaybackPilsner Bo...|[paybackpilsner,

In [131]:
from pyspark.ml.feature import StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')



In [132]:
score_pipeline = Pipeline(stages=[
  pos_neg_to_num
])

In [133]:
prep_pipeline = score_pipeline.fit(score_df)
final_score_df = prep_pipeline.transform(score_df)

In [134]:
final_score_df.show()

+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------+-----+
|beer_id|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          Final_text|            features|   class|label|
+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------+-----+
| 271781|         4.0|          4.0|          4.0|        4.25|            4.0|         4.03|MotorbreathImperi...|[motorbreathimper...|Positive|  0.0|
| 184647|         4.0|          4.0|          3.5|         3.5|           3.75|          3.7|PaybackPilsner Bo...|[paybackpilsner, ...|Positive|  0.0|
| 184647|         4.0|          4.0|         4.25|        3.75|            4.0|         4.08|PaybackPilsner Bo...|[paybackpilsner, ...|Positive|  0.0|
| 184647|        3.25|         3.25|          3.5|         3.5|            3.5|         3.43|P

In [135]:
training, testing = final_score_df.randomSplit([0.7, 0.3])

In [136]:
vecs_training = w2v_loaded_model.transform(training)

In [137]:
vecs_training.show()

+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------+-----+--------------------+
|beer_id|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          Final_text|            features|   class|label|               model|
+-------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------+-----+--------------------+
| 100025|         4.5|         4.25|          4.5|        4.25|           4.25|         4.37|NvUPajamaParty Am...|[nvupajamaparty, ...|Positive|  0.0|[-0.0592176561766...|
| 100027|         3.0|         3.75|          4.0|         4.5|            4.0|         3.93|CrookedStaveOrigi...|[crookedstaveorig...|Positive|  0.0|[-0.0767600935318...|
| 100027|        3.25|          3.5|         3.75|        3.75|           3.75|         3.66|CrookedStaveOrigi...|[crookedstaveorig...|Posit

In [138]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(
    featuresCol = 'model', 
    labelCol = 'label',
    maxIter=10
)
lrModel = lr.fit(vecs_training)

In [139]:
lrModel.coefficients

DenseVector([-0.7906, -5.0067, -4.111, 0.1153, 1.4114, -8.19, -1.7127, -7.4499, 0.037, -0.3231, -0.9709, 2.927, -4.5678, 0.1553, -1.0436, -0.9436, 2.65, -2.7767, 0.177, 3.9497, 1.0301, -0.3838, -0.5297, 3.28, -2.525, 0.5515, 0.047, 1.6923, 3.8833, -7.2038, -3.8247, 1.6487, -4.085, -1.1063, -0.4885, 4.533, 0.2058, -1.221, 0.3975, 0.8063, -1.8955, 4.3636, -2.3265, 4.9642, -1.5338, 2.9412, 3.3978, 4.4196, 5.4482, -1.3596, 1.8558, -6.9468, -2.4115, -1.1507, 4.8384, 0.2485, 2.7739, 1.1379, 3.7788, -4.3628, -2.3959, -4.7326, 4.5702, -4.5326, -4.6736, 3.0256, -0.7662, 2.5552, 1.9247, -0.6979, -1.7802, -1.1397, 3.3484, -0.6827, -3.0076, -3.2312, -0.2341, -0.7933, -0.3217, 1.8675, -3.892, -0.5791, 0.6517, -0.5024, 0.2726, -2.966, -1.4472, -4.0553, -2.0125, 2.6572, -2.1944, 3.6951, -0.1167, 3.7871, 5.3447, -8.0631, -2.2381, -0.5711, 3.8527, 0.2104])

In [141]:
vecs_testing = w2v_loaded_model.transform(testing)

predictions = lrModel.transform(vecs_testing)

In [157]:
prediction_df = predictions.select('class', 'label', 'prediction','score_numeric', 'Final_text','probability', 'rawPrediction')
prediction_df.show(20, truncate= False)

+--------+-----+----------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [164]:
nonmatch_df = prediction_df.filter('label != prediction')
nonmatch_df.count()

81330

In [165]:
nonmatch_df.show(30, truncate= False)

+--------+-----+----------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [143]:
# # Use the Class Evaluator for a cleaner description
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# acc_eval = MulticlassClassificationEvaluator()
# acc = acc_eval.evaluate(predictions)
# print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.847797


In [101]:
# Create a df that had only the columns needed, leaving final text in this version for clustering later
# compare_df = final_df.select('score_numeric', 'overall_numeric','features', 'Final_text')